In [1]:
import torch
import time as t
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Prepare data

train_samples = datasets.ImageFolder('data/training', transforms.ToTensor())
test_samples = datasets.ImageFolder('data/testing', transforms.ToTensor())

In [3]:
# Load data
train_set = DataLoader(train_samples, batch_size=100, shuffle=True, num_workers=0)
test_set = DataLoader(test_samples, batch_size=100, shuffle=False, num_workers=0)

In [4]:
class Cnn(nn.Module):
    
    def __init__(self):
        
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5, padding=1)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5, padding=1)
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self,x):
        x = F.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 5*5*50)
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [5]:
# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create the Cnn
model = Cnn().to(device)

# Create the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,  momentum=0.9, weight_decay=0)

# Establish our loss funcion (NLLLoss needs log_softmax outputs)
criterion = nn.NLLLoss()

In [6]:
def train(model, optimizer, criterion):
    
    model.train() # training mode
    
    running_loss = 0
    running_corrects = 0
    
    for x,y in train_set:
        
        x=x.to(device)
        y=y.to(device)
        
        optimizer.zero_grad() # make the gradients 0 
        output = model(x) # forward pass
        _, preds = torch.max(output, 1)
        
        loss = criterion(output, y) # calculate the loss value
        
        loss.backward() # compute the gradients
        optimizer.step() # uptade network parameters 
                
        # statistics 
        running_loss += loss.item() * x.size(0)
        running_corrects += torch.sum(preds==y).item()
            
    epoch_loss = running_loss / len(train_samples) # mean epoch loss 
    epoch_acc = running_corrects / len(train_samples) # mean epoch accuracy
    
    return epoch_loss, epoch_acc

In [7]:
def test(model, optimizer, criterion):
    

    model.eval() # evaluation mode
    
    running_loss = 0
    running_corrects = 0
    
    # we do not need to compute the gradients in eval mode
    with torch.no_grad(): 

        for x,y in test_set:
            
            x=x.to(device)
            y=y.to(device)
        
            output = model(x) # forward pass
            _, preds = torch.max(output, 1)

            loss = criterion(output, y) # calculate the loss value
           
            # statistics 
            running_loss += loss.item() * x.size(0)
            running_corrects += torch.sum(preds==y).item()
            
    epoch_loss = running_loss / len(test_samples) # mean epoch loss 
    epoch_acc = running_corrects / len(test_samples) # mean epoch accuracy
    
    
    return epoch_loss, epoch_acc

In [8]:
for epoch in range(10):

    start = t.time()

    train_loss, train_acc = train(model, optimizer, criterion)
    print('-' * 72)
    print('| End of epoch: {:3d} | Time: {:.2f}s | Train loss: {:.3f} | Train acc: {:.3f} |'
          .format(epoch + 1, t.time() - start, train_loss, train_acc))
    
    test_loss, test_acc = test(model, optimizer, criterion)
    print('-' * 72)
    print('| End of epoch: {:3d} | Time: {:.2f}s | Test loss: {:.3f} | Test acc: {:.3f} |'
          .format(epoch + 1, t.time() - start, test_loss, test_acc))

------------------------------------------------------------------------
| End of epoch:   1 | Time: 70.15s | Train loss: 0.373 | Train acc: 0.963 |
------------------------------------------------------------------------
| End of epoch:   1 | Time: 74.22s | Test loss: 6.825 | Test acc: 0.101 |
------------------------------------------------------------------------
| End of epoch:   2 | Time: 149.21s | Train loss: 0.306 | Train acc: 0.943 |
------------------------------------------------------------------------
| End of epoch:   2 | Time: 153.21s | Test loss: 6.810 | Test acc: 0.101 |


KeyboardInterrupt: 